In [52]:
import wget
import urllib.parse
import os
import requests
import tarfile
import warnings
import regex
import yaml

import pandas as pd

from tqdm import tqdm
from lxml import html

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

# Webscraping

## Get URLS

In [12]:
def webscrapping_tar(page="http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"):
    """Web scraping
    1. Get url to all href elements
    2. Check if it is a .tgz file
    3  Return list with the urls
    """
    page = requests.get(page)
    webpage = html.fromstring(page.content)
    files=[]
    for element in webpage.xpath('//a/@href'):
        if ".tgz" in element:
            files.append(element)
    return files
files = webscrapping_tar()

In [13]:
print("The number of files is {}".format(len(files)))

The number of files is 6244


In [17]:
# create the data folder
#! mkdir data

# Download files

In [19]:
def download_tar(save_in_folder="data",list_urls=None,base_url="http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"):
    """ Function to download files
    1. Gets list with the URLS
    2. Downloads all the files
    3. Stores the .tgz files in the folder data
    """
    for url in tqdm(list_urls):
        url_file = url
        url=urllib.parse.urljoin(base_url, url_file)
        path_file = os.path.join("data",url_file)
        wget.download(url, path_file)
    
download_tar(list_urls=files)

100%|██████████| 6244/6244 [1:59:24<00:00,  1.15s/it]


## Unzip files

In [20]:
def extract_file(fname=None, destination="extracted_data/"):
    """ Function to unzip a folder
    1. Get folder name and checks type
    2. Returns warning if the format is not supported
    3. The unzipe data is passed to the folder extracted_data
    """
    if (fname.endswith("tar.gz") or fname.endswith("tgz")):
        tar = tarfile.open(fname, "r:gz")
        tar.extractall(path=destination)
        tar.close()
    elif (fname.endswith("tar")):
        # dont use in this case but keept it
        tar = tarfile.open(fname, "r:")
        tar.extractall(path=destination)
        tar.close()
    else:
        warnings.warn("The file was not extracted, problem with file format for file {}".format(fname))
#extract_file("data/1028-20100710-hne.tgz")

In [21]:
def extract_all_files(folder="data"):
    """ Function that loops over all folder in a folder and unzips them
    1. Checks for folder in a folder
    2. Unzips all the folders
    """
    dirs= os.listdir(folder)
    for tar in tqdm(dirs):
        path = os.path.join(folder,tar)
        extract_file(path)
extract_all_files()

100%|██████████| 6244/6244 [05:37<00:00, 18.52it/s]


## Validate the number of files

In [85]:
!(cd extracted_data && find . -type f -printf '%f\n' | sed -r -n 's/.+(\..*)$/\1/p' | sort | uniq -c | sort -bn)


      1 .prompt
      1 .prompts
      1 .wav-bak
      1 .wav-omit
      1 .wav-skip
      2 .text
      2 .wav-split
      7 .wav-original
   5704 .txt
   8825 .flac
  86656 .wav


Based upon the state information in the assignment it was assumed to be 62,440 audio samples. However there are at least more recodrings than that avilable. Further investigateion is needed later on. 

# Extract the text

In [39]:

def extract_text_info(path):
    """ Function to extract the infomration from the text files
    1. Opens text file
    2. Using regex(fuzzy) checks for the features of interest: gender,age_group,dialect,language
    3. Returns the extracted data as a dict
    """
    text_file = open(path, "r")
    lines = text_file.readlines()
    regex_expressions = yaml.load(open('extract.yaml'))
    respons=dict()
    # If I cant find it in the text i want to return None for the key
    respons['Gender']=None
    respons['Age_Range']=None
    respons['Language']=None
    respons['Dialect']=None
    for field, value in regex_expressions.items():
        for line in lines: 
            match=regex.search(regex_expressions.get(field).get('key'),line)
            if match:      
                respons[field]=match.group(1)
    return respons
#path = "/Users/Niklas/Documents/test/extracted_data/1028-20100710-hne/etc/README"
#extract_text_info(path)

# Extract info from all the unziped files


In [43]:
def label_and_path(base_folder = "extracted_data",output_file="data.csv"):
    """ Function that loops over all the unziped folders and extracts the information
    1. Gets the folder where the extracted data is
    2. Loops over all the folders in the folder
    3. Foreach of the folders the information is extracted also the path to the sound files
    4. Stores the information in a pandas dataframe that is then saved as a csv
    """
    dirs= os.listdir(base_folder)
    df = pd.DataFrame()
    paths = []
    age_range = []
    language = []
    dialect = []
    gender = []

    for folder in tqdm(dirs):
        if not folder.endswith(".DS_Store"):
            path = os.path.join(base_folder,folder)
            path = os.path.join(os.getcwd(),path)
            complete_path = os.path.join(path,"etc/README")
            if os.path.isfile(complete_path):
                ans = extract_text_info(complete_path)
                sound_path = os.path.join(path,"wav")
                if os.path.isdir(sound_path):
                    for file in os.listdir(sound_path):
                        sound_file_path = os.path.join(sound_path,file)
                        paths.append(sound_file_path)
                        age_range.append(ans['Age_Range'])
                        language.append(ans['Language'])
                        dialect.append(ans['Dialect'])
                        gender.append(ans['Gender'])
                        # print(path)
                        # I know here that i should go into wav and add the sound files 
                        # I also know that i should go into etc to get the information about the files
                        # DO the magic here
                else:
                    print("Missing sound files for {}".format(sound_path))
            else:
                print("Missing label for file {}".format(complete_path))
    df['path'] = paths
    df['gender'] = gender
    df['language'] = language
    df['age_range'] = age_range
    df['dialect'] = dialect
    df.to_csv(output_file)
label_and_path()

  1%|▏         | 86/6244 [00:00<00:14, 423.64it/s]

Missing sound files for /home/niklas_sven_hansson/test/extracted_data/ralfherzog-20080203-en51/wav
Missing sound files for /home/niklas_sven_hansson/test/extracted_data/bjb-20120204/wav
Missing sound files for /home/niklas_sven_hansson/test/extracted_data/ralfherzog-20080312-en59/wav


  3%|▎         | 218/6244 [00:00<00:14, 429.25it/s]

Missing sound files for /home/niklas_sven_hansson/test/extracted_data/ralfherzog-20070923-en8/wav
Missing label for file /home/niklas_sven_hansson/test/extracted_data/jaiger-12032006-6/etc/README


  5%|▌         | 342/6244 [00:00<00:14, 420.76it/s]

...

 96%|█████████▋| 6010/6244 [00:14<00:00, 421.38it/s]

Missing sound files for /home/niklas_sven_hansson/test/extracted_data/bjb-20120124/wav
Missing sound files for /home/niklas_sven_hansson/test/extracted_data/ralfherzog-20080309-en56/wav
Missing sound files for /home/niklas_sven_hansson/test/extracted_data/bjb-20120409/wav
Missing sound files for /home/niklas_sven_hansson/test/extracted_data/ralfherzog-20080203-en50/wav
Missing sound files for /home/niklas_sven_hansson/test/extracted_data/ralfherzog-20071214-en38/wav
Missing sound files for /home/niklas_sven_hansson/test/extracted_data/ralfherzog-20070831-en1/wav


100%|█████████▉| 6233/6244 [00:14<00:00, 422.03it/s]

Missing sound files for /home/niklas_sven_hansson/test/extracted_data/ralfherzog-20071126-en24/wav
Missing sound files for /home/niklas_sven_hansson/test/extracted_data/bjb-20120310/wav
Missing sound files for /home/niklas_sven_hansson/test/extracted_data/ralfherzog-20070830_vf25/wav


100%|██████████| 6244/6244 [00:14<00:00, 421.95it/s]


In [45]:
df = pd.read_csv("data.csv",index_col=0)
df.shape

(82893, 5)

In [53]:
df

,path,gender,language,age_range,dialect
0,/home/niklas_sven_hansson/test/extracted_data/phr0663r-20090922-jqj/wav/a0492.wav,Male,EN,Adult,British English
1,/home/niklas_sven_hansson/test/extracted_data/phr0663r-20090922-jqj/wav/a0493.wav,Male,EN,Adult,British English
2,/home/niklas_sven_hansson/test/extracted_data/phr0663r-20090922-jqj/wav/a0487.wav,Male,EN,Adult,British English
3,/home/niklas_sven_hansson/test/extracted_data/phr0663r-20090922-jqj/wav/a0488.wav,Male,EN,Adult,British English
4,/home/niklas_sven_hansson/test/extracted_data/phr0663r-20090922-jqj/wav/a0495.wav,Male,EN,Adult,British English
5,/home/niklas_sven_hansson/test/extracted_data/phr0663r-20090922-jqj/wav/a0490.wav,Male,EN,Adult,British English
6,/home/niklas_sven_hansson/test/extracted_data/phr0663r-20090922-jqj/wav/a0496.wav,Male,EN,Adult,British English
7,/home/niklas_sven_hansson/test/extracted_data/phr0663r-20090922-jqj/wav/a0494.wav,Male,EN,Adult,British English
8,/home/niklas_sven_hansson/test/extracted_data/phr0663r-20090922-jqj/wav/a0489.wav,Male,EN,Adult,British English
9,/home/niklas_sven_hansson/test/extracted_data/phr0663r-20090922-jqj/wav/a0491.wav,Male,EN,Adult,British English


## Check the file information

In [47]:
# Observations where the language is stated to be english
df[df["language"]=="EN"].shape

(59027, 5)

In [49]:
# Count of the unique values of english
df['language'].value_counts()

EN    59027
Name: language, dtype: int64

In [75]:
# Sum of all dialects
df['dialect'].value_counts()

American English      32187
British English       9167 
European English      8320 
Other                 4203 
Canadian English      3128 
Indian English        2645 
Australian English    2360 
General American      1822 
New Zealand           819  
South African         783  
other                 628  
New York              551  
Irish English         329  
unknown               173  
Please Select         158  
Northern Irish        155  
Mid                   150  
California            94   
British Received      70   
West                  64   
european english      49   
English English       39   
Western               31   
European              24   
non                   10   
Westdeutschland       10   
Asian accent          10   
America English       10   
Español España        10   
Name: dialect, dtype: int64

In [67]:
df["gender"].value_counts()

Male         52760
male         6837 
Female       3824 
unknown      193  
Please       189  
female       157  
adult        10   
make         10   
Masculino    10   
Weiblich     10   
Name: gender, dtype: int64

In [70]:
sum(df["gender"].value_counts())-189-193

63618

## Only select the once that actually have a label of interest

In [86]:
test = df[((df["gender"] =='Male' )|(df["gender"]=='male')|
          (df["gender"]=='Female')|(df["gender"]=='female'))  & (df["language"]=="EN")]
test.shape

(58615, 5)

## Validat the paths

In [65]:
def test_files_exsist(df=None):
    for file in tqdm(df['path']):
        if not os.path.isfile(file):
            warning.warn("This path is wrong {}".format(file))

def test_files_unique():
    if not df.shape[0]==len(df['path'].unique()):
        print("Some rows are duplicates")
test_files_exsist(df)
test_files_unique()

100%|██████████| 82893/82893 [00:00<00:00, 226527.18it/s]


data	  extracted_data  ouput.csv	     sound_feature.R
data.csv  extract.yaml	  sound_feature_2.R  Untitled.ipynb
